In [30]:
import math
import pandas as pd
import plotly
import plotly.express as px
from matplotlib import pyplot as plt

from cadCAD.configuration.utils import config_sim

from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

In [31]:
import networkx as nx
import numpy as np
from cadCAD.configuration import Experiment
from numpy import random
from cadCAD import configs
del configs[:] # Clear any prior configs

experiment = Experiment()

# Setup / Preparatory Steps

## Initializing Agents Network

<center>
<img src="./images/FinTech.jpg"
     alt="Diff spec"
     style="width: 100%" />
</center>

<center>
<img src="./images/FinTech_CL.jpg"
     alt="Diff spec"
     style="width: 100%" />
</center>

In [32]:
initial_state = {
    'companies': 15, # number of companies
    'treasury': 0, # in usd
    'expenses': 0, # in usd
    'net_income': 0,
    'external_kyb': 0,
    'external_exchange':0,
    'external_liquidity':0,
    'external_fiat':0,
    'external_wallet':0
}

system_params = {
    'increase_rate': [0.05, 0.07],
    'decrease_rate': [0.003, 0.005],
    'fees': [0.03, 0.015],
    'buy_rate': [0.05, 0.08],
    'sell_rate': [0.10, 0.12],
    'expenses_kyb': [2500.0],
    'expenses_exchange_provider': [0.0003],
    'expenses_liquidity_providers': [0.001],
    'expenses_fiat_gateway': [0.025], 
    'expenses_wallet_provider': [2000.0]
}

## 1. Logic

\begin{align}
\large population_t &\large= population_{t-1} + {\Delta population} \quad \textrm{(users)} \tag{1} \\
\large treasury_t &\large= treasury_{t-1} + {\Delta treasury} * fees \quad \textrm{(income from fees)} \tag{2}
\end{align}

where the rate of change ($\Delta$) is:
\begin{align}
\large {\Delta population} &\large= \alpha * population_{t-1} - \epsilon * population_{t-1} \quad \textrm{(new users/month)} \\
\large {\Delta treasury} &\large= \beta * population_{t-1} + \gamma * population_{t-1} \quad \textrm{(income/month)} \\
\end{align}
where:

$
\begin{align}
\alpha: \quad &\textrm{'increase_rate'}\\
\epsilon: \quad &\textrm{'decrease_rate'}\\
\beta: \quad &\textrm{'buy_rate'}\\
\gamma: \quad &\textrm{'sell_rate'}\\
\end{align}
$

In [33]:
def p_new_agent(params, 
                substep, 
                state_history, 
                previous_state):
    p = previous_state['companies']
    increase_rate = params['increase_rate']
    t = previous_state['timestep']
    rand = np.random.rand()
    
    if rand < increase_rate:
        increase_rate = increase_rate
    else: 
        increase_rate = 0
    value = p * increase_rate
    return {'add_to_company': value}
    

def p_remove_agent(params, 
                   substep, 
                   state_history, 
                   previous_state):
    p = previous_state['companies']
    decrease_rate = params['decrease_rate']
    t = previous_state['timestep']
    division =  t % 31  == 0
    if division == True:
        decrease_rate = decrease_rate
    else: 
        decrease_rate = 0
    value = p * decrease_rate
    return {'add_to_company': -1.0 * value}

    
def p_sales(params, 
            substep, 
            state_history, 
            previous_state):
    p = float(previous_state['companies'])
    buys = params['buy_rate']
    sells = params['sell_rate']
    erMargin = random.uniform(0, 0.15)
    const_fee = params['fees']
    interaction = 0.5
    random_amount_to_buy = random.randint(2000, 10000)
    random_amount_to_sell = random.randint(5000, 20000)
    rd = random.uniform(0, 0.25)
    timestep = previous_state['timestep']
    division =  timestep % 2  == 0
    if (interaction > random.random()) & (division == True):
        interacting_buy = (p * (buys-erMargin)) * random_amount_to_buy
    else:
        interacting_buy = 0
    
    if (interaction > np.random.random()) and (division == True):
        interacting_sell = (p * (sells - erMargin)) * random_amount_to_sell
    else:
        interacting_sell = 0
    sales_total = (interacting_buy + interacting_sell) * const_fee
    return{'total_sales': sales_total}

    
    
def p_expenses(params, 
               substep, 
               state_history, 
               previous_state):
    p = float(previous_state['companies'])
    t = previous_state['treasury']
    kyb = params['expenses_kyb']
    exchange_provider = params['expenses_exchange_provider']
    liquidity_providers = params['expenses_liquidity_providers']
    fiat_gateway = params['expenses_fiat_gateway']
    wallet_provider = params['expenses_wallet_provider']
    timestep = previous_state['timestep']
    division =  timestep % 31  == 0
    if division == True and p > 1000:
        expenses = kyb + wallet_provider + (t * exchange_provider) + (t * fiat_gateway) + (t * liquidity_providers)
    elif division == True and p < 1000:
        kyb_l = 5000
        wallet_provider_l = 3000
        expenses = kyb_l + wallet_provider_l + (t * exchange_provider) + (t * fiat_gateway) + (t * liquidity_providers)
    else:
        expenses = 0
    expenses_value = expenses
    return{'total_expenses': expenses_value}

In [34]:
def p_kyb(params, 
               substep, 
               state_history, 
               previous_state):
    p = float(previous_state['companies'])
    t = previous_state['treasury']
    timestep = previous_state['timestep']
    division =  timestep % 31  == 0
    kyb = params['expenses_kyb']
    if division == True and p > 100:
        expenses = kyb
    elif division == True and p < 100:
        expenses = 3000
    else:
        expenses = 0
    expenses_kyb = expenses
    return{'total_e_kyb': expenses_kyb}

In [35]:
def p_exchange(params, 
               substep, 
               state_history, 
               previous_state):
    t = previous_state['treasury']
    timestep = previous_state['timestep']
    division =  timestep % 31  == 0
    exchange_provider = params['expenses_exchange_provider']

    if division == True:
        expenses = exchange_provider * t
    else:
        expenses = 0
    expenses_value = expenses
    return{'total_e_exchange': expenses_value}

In [36]:
def p_wallet(params, 
               substep, 
               state_history, 
               previous_state):
    p = float(previous_state['companies'])
    t = previous_state['treasury']
    timestep = previous_state['timestep']
    division =  timestep % 31  == 0
    wallet_provider = params['expenses_wallet_provider']

    if division == True:
        expenses = wallet_provider
    elif division == True and p > 1000:
        expenses = 2000
    else:
        expenses = 0
    expenses_wallet = expenses
    return{'total_e_wallet': expenses_wallet}

In [37]:
def p_liquididty(params, 
               substep, 
               state_history, 
               previous_state):
    t = previous_state['treasury']
    timestep = previous_state['timestep']
    division =  timestep % 31  == 0
    liquidity_providers = params['expenses_liquidity_providers']
    if division == True:
        expenses = t * liquidity_providers
    else:
        expenses = 0
 
    expenses_liquidity = expenses
    return{'total_e_liquidity': expenses_liquidity}

In [38]:
def p_fiat(params, 
               substep, 
               state_history, 
               previous_state):
    t = previous_state['treasury']
    timestep = previous_state['timestep']
    division =  timestep % 31  == 0
    fiat_gateway = params['expenses_fiat_gateway']

    if division == True:
        expenses = t * fiat_gateway
    else:
        expenses = 0
    expenses_fiat = expenses
    return{'total_e_fiat': expenses_fiat}

## 2. State Update

In [39]:

def s_treasury(params, substep, state_history, previous_state, policy_input):
    treasury = previous_state['treasury'] + policy_input['total_sales']
    return ('treasury', max(treasury, 0))

def s_expenses(params, substep, state_history, previous_state, policy_input):
    expenses =previous_state['expenses'] + policy_input['total_expenses']
    return ('expenses', max(expenses, 0))

def s_companies(params, substep, state_history, previous_state, policy_input):
    companies = previous_state['companies']
    add_to_company = policy_input['add_to_company']
    updated_companies = companies + add_to_company
    return 'companies', max(math.ceil(updated_companies), 0)

def s_net_income(params, substep, state_history, previous_state, policy_input):
    net_income = previous_state['treasury'] - previous_state['expenses']
    return ('net_income', max(net_income, 0))


In [40]:
def s_kyb(params, substep, state_history, previous_state, policy_input):
    expenses_kyb = previous_state['external_kyb'] + policy_input['total_e_kyb']
    return ('external_kyb', max(expenses_kyb, 0))

In [41]:
def s_exchange(params, substep, state_history, previous_state, policy_input):
    expenses_exchange = previous_state['external_exchange'] + policy_input['total_e_exchange']
    return ('external_exchange', max(expenses_exchange, 0))

In [42]:
def s_wallet(params, substep, state_history, previous_state, policy_input):
    expenses_wallet = previous_state['external_wallet'] + policy_input['total_e_wallet']
    return ('external_wallet', max(expenses_wallet, 0))

In [43]:
def s_liquidity(params, substep, state_history, previous_state, policy_input):
    expenses_liquidity = previous_state['external_liquidity'] + policy_input['total_e_liquidity']
    return ('external_liquidity', max(expenses_liquidity, 0))

In [44]:
def s_fiat(params, substep, state_history, previous_state, policy_input):
    expenses_fiat = previous_state['external_fiat'] + policy_input['total_e_fiat']
    return ('external_fiat', max(expenses_fiat, 0))

## 3. Partial State Update

In [45]:
partial_state_update_blocks = [
    {
        'policies': {
            'total_expenses': p_expenses,
            'total_e_kyb': p_kyb,
            'total_e_exchange': p_exchange,
            'total_e_wallet': p_wallet,
            'total_e_liquididty': p_liquididty,
            'total_e_fiat': p_fiat,
            'sales': p_sales,
            'p_new_agents': p_new_agent, 
            'p_remove_agents': p_remove_agent
        },
        'variables': {
            'companies': s_companies,
            'treasury': s_treasury,
            'expenses': s_expenses,
            'net_income': s_net_income,
            'external_kyb': s_kyb,
            'external_exchange': s_exchange,
            'external_liquidity': s_liquidity,
            'external_fiat': s_fiat,
            'external_wallet': s_wallet,
        }
    }
]

## 4. Configuration

In [46]:
MONTE_CARLO_RUNS = 5
SIMULATION_TIMESTEPS = 365

sim_config = config_sim({
    'N': MONTE_CARLO_RUNS,
    'T': range(SIMULATION_TIMESTEPS),
    'M': system_params
})


In [47]:
del configs[:] # Clear any prior configs

In [48]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

## 4. Execution

In [49]:
exec_mode = ExecutionMode()
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 2
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (365, 10, 5, 9)
Execution Method: local_simulations
SimIDs   : [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
SubsetIDs: [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
Ns       : [0, 1, 2, 3, 4, 0, 1, 2, 3, 4]
ExpIDs   : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Execution Mode: parallelized
Total execution time: 0.43s


In [50]:
simulation_result = pd.DataFrame(raw_result)
simulation_result

,companies,treasury,expenses,net_income,external_kyb,external_exchange,external_liquidity,external_fiat,external_wallet,simulation,subset,run,substep,timestep
0,15,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,0.0,0,0,1,0,0
1,15,0.000000,8000.000000,0,3000.0,0.000000,0.000000,0.000000,2000.0,0,0,1,1,1
2,15,0.000000,8000.000000,0,3000.0,0.000000,0.000000,0.000000,2000.0,0,0,1,1,2
3,15,0.000000,8000.000000,0,3000.0,0.000000,0.000000,0.000000,2000.0,0,0,1,1,3
4,15,0.000000,8000.000000,0,3000.0,0.000000,0.000000,0.000000,2000.0,0,0,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655,100,42281.875546,100931.058425,0,33000.0,56.247815,187.492716,4687.317894,24000.0,1,1,5,1,361
3656,100,42281.875546,100931.058425,0,33000.0,56.247815,187.492716,4687.317894,24000.0,1,1,5,1,362
3657,100,42406.121569,100931.058425,0,33000.0,56.247815,187.492716,4687.317894,24000.0,1,1,5,1,363
3658,100,42406.121569,100931.058425,0,33000.0,56.247815,187.492716,4687.317894,24000.0,1,1,5,1,364


## 5. Sweep Data Parameters

In [51]:
# Convert raw results to a Pandas DataFrame
df = pd.DataFrame(raw_result)

# Insert cadCAD parameters for each configuration into DataFrame
for config in configs:
    # Get parameters from configuration
    parameters = config.sim_config['M']
    # Get subset index from configuration
    subset_index = config.subset_id
    
    # For each parameter key value pair
    for (key, value) in parameters.items():
        # Select all DataFrame indices where subset == subset_index
        dataframe_indices = df.eval(f'subset == {subset_index}')
        # Assign each parameter key value pair to the DataFrame for the corresponding subset
        df.loc[dataframe_indices, key] = value

df.head(100)

,companies,treasury,expenses,net_income,external_kyb,external_exchange,external_liquidity,external_fiat,external_wallet,simulation,...,increase_rate,decrease_rate,fees,buy_rate,sell_rate,expenses_kyb,expenses_exchange_provider,expenses_liquidity_providers,expenses_fiat_gateway,expenses_wallet_provider
0,15,0.000000,0.000000,0,0.0,0.000000,0.00000,0.000000,0.0,0,...,0.05,0.003,0.03,0.05,0.1,2500.0,0.0003,0.001,0.025,2000.0
1,15,0.000000,8000.000000,0,3000.0,0.000000,0.00000,0.000000,2000.0,0,...,0.05,0.003,0.03,0.05,0.1,2500.0,0.0003,0.001,0.025,2000.0
2,15,0.000000,8000.000000,0,3000.0,0.000000,0.00000,0.000000,2000.0,0,...,0.05,0.003,0.03,0.05,0.1,2500.0,0.0003,0.001,0.025,2000.0
3,15,0.000000,8000.000000,0,3000.0,0.000000,0.00000,0.000000,2000.0,0,...,0.05,0.003,0.03,0.05,0.1,2500.0,0.0003,0.001,0.025,2000.0
4,15,0.000000,8000.000000,0,3000.0,0.000000,0.00000,0.000000,2000.0,0,...,0.05,0.003,0.03,0.05,0.1,2500.0,0.0003,0.001,0.025,2000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20,1999.196465,32103.863438,0,12000.0,1.184754,3.94918,98.729504,8000.0,0,...,0.05,0.003,0.03,0.05,0.1,2500.0,0.0003,0.001,0.025,2000.0
96,20,1999.196465,32103.863438,0,12000.0,1.184754,3.94918,98.729504,8000.0,0,...,0.05,0.003,0.03,0.05,0.1,2500.0,0.0003,0.001,0.025,2000.0
97,20,1906.952867,32103.863438,0,12000.0,1.184754,3.94918,98.729504,8000.0,0,...,0.05,0.003,0.03,0.05,0.1,2500.0,0.0003,0.001,0.025,2000.0
98,20,1906.952867,32103.863438,0,12000.0,1.184754,3.94918,98.729504,8000.0,0,...,0.05,0.003,0.03,0.05,0.1,2500.0,0.0003,0.001,0.025,2000.0


## 6. Analysis

In [52]:
px.line(
    df,
    x='timestep', # Variable on the horizontal axis
    y=['companies', 'treasury', 'expenses', 'net_income'], # Variables on the vertical axis
    facet_row='run',
    facet_col='subset',
    
    height=800,
)

In [53]:
px.line(
    df,
    x='timestep', # Variable on the horizontal axis
    y=['companies', 'treasury', 'external_kyb', 'external_exchange', 'external_liquidity', 'external_fiat', 
'external_wallet', 'expenses'], # Variables on the vertical axis
    facet_row='run',
    facet_col='subset',
    title='Detailed Expenses',
    height=800,
)

\begin{align}
\large companies_t &\large= companies_{t-1} + {\Delta companies} \quad \textrm{(users)} \tag{1} \\
\large treasury_t &\large= treasury_{t-1} + ({\Delta treasury} * fees \quad \textrm{(income from fees)} \tag{2}
\end{align}

where the rate of change ($\Delta$) is:
\begin{align}
\large {\Delta companies} &\large= \alpha * companies_{t-1} - \epsilon * companies_{t-1} \quad \textrm{(new users/month)} \\
\large {\Delta treasury} &\large= \beta * companies_{t-1} + \gamma * companies_{t-1} \quad \textrm{(income/month)} \\
\end{align}
where:

$
\begin{align}
\alpha: \quad &\textrm{'increase_rate' = 5% and 7%}\\
\epsilon: \quad &\textrm{'decrease_rate' = 0.03% and 0.05%}\\
\beta: \quad &\textrm{'buy_rate'}\\
\gamma: \quad &\textrm{'sell_rate'}\\
\end{align}
$

In [54]:
pd.options.plotting.backend = "plotly"

In [55]:
df.plot(
    kind='line',
    x='expenses', # Variable on the horizontal axis
    y='companies', # Variables on the vertical axis
    color='increase_rate' # Color by dt
)

In [56]:
# Filter data frame so that we get the data from the 1st MC run
filtered_df = df.query('run == 1')

filtered_df.plot(
    kind='line',
    x='expenses', # Variable on the horizontal axis
    y='companies', # Variables on the vertical axis
    color='increase_rate' # Color by prey_death_parameter
)

In [57]:
filtered_df = df.query('decrease_rate == 0.003 & run == 1')

filtered_df.plot(
    kind='line',
    x='companies', # Variable on the horizontal axis
    y='treasury', # Variables on the vertical axis
    color='increase_rate' # Color by prey_death_parameter
)

In [58]:
filtered_df = df.query('decrease_rate == 0.005 & run == 1')

filtered_df.plot(
    kind='line',
    x='companies', # Variable on the horizontal axis
    y='treasury', # Variables on the vertical axis
    color='increase_rate' # Color by prey_death_parameter
)